In [1]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
def colour_histogram(patch,number=8):
    histogram = np.zeros((3,number),dtype=int)
    interval = 256//number
    for channel in range(3):
        values = patch[:,:,channel]
        for row in values:
            for v in row:
                binn = v//interval
                histogram[channel,binn] += 1
    vector = np.concatenate((histogram[0],histogram[1],histogram[2]))
    return vector

In [3]:
def distance(item, mean):
    sum_ = 0.0
    dim = len(item)
    for j in range(dim):
        sum_ += (item[j] - mean[j]) ** 2
    return np.sqrt(sum_)

In [4]:
path = 'C:/Users/Tirthashree/Documents/MTech/Notes/DLA/Assignments/Group20/Classification/Image_Group20/train/'
os.chdir(path)

train_images = []
for root, dirs, filenames in os.walk(path):
    for folder in dirs:
        for file in  os.listdir(os.path.join(path,folder)):
            train_images.append(os.path.join(path,folder,file))
# print(train_images)
print(len(train_images))

path = 'C:/Users/Tirthashree/Documents/MTech/Notes/DLA/Assignments/Group20/Classification/Image_Group20/test/'
os.chdir(path)

test_images = []
for root, dirs, filenames in os.walk(path):
    for folder in dirs:
        for file in  os.listdir(os.path.join(path,folder)):
            test_images.append(os.path.join(path,folder,file))
print(len(test_images))
# for file in  os.listdir(path):
#     train_images.append(os.path.join(path,file))

150
150


In [5]:
# Extracting 24-D feature vectors for training data
feature_vectors_train = []
# count_train = 0
count_patches_train = []
for img in train_images: # iterating over every image in training set
#     count_train += 1 
#     print(count_train)
    image = cv2.imread(img)  # reading the image
    shape = image.shape  # finding shape of the image
    size_1 = int(np.ceil(shape[0]/32)*32)  # finding the nearest integer to form 32x32 patches, row wise
    size_2 = int(np.ceil(shape[1]/32)*32)  # finding the nearest integer to form 32x32 patches, column wise
    rf = [0,0]   # repeating_factor for padding image
    rf[0] = (size_1-shape[0])
    rf[1] = (size_2-shape[1])
    # padding the image by border pixel values on each side
    final_image = cv2.copyMakeBorder(image, rf[0]//2, rf[0]-rf[0]//2, rf[1]//2,rf[1]-rf[1]//2, cv2.BORDER_REPLICATE)
    patches = np.ndarray(shape=(size_1//32,size_2//32,32,32,3), dtype=np.uint8) # to store the patches
    # forming patches
    for i in range(size_1//32): # storing the patches
        for j in range(size_2//32):
            patches[i,j] = final_image[32*i:32*(i+1),32*j:32*(j+1)]
    # colour histogram for every patch
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            feature_vectors_train.append(colour_histogram(patches[i,j]))
    count_patches_train.append((size_1//32)*(size_2//32)) # storing the number of patches for each image
feature_vectors_train = np.array(feature_vectors_train)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [6]:
print('Number of feature vectors: ',len(feature_vectors_train))
print('Number of patches: ',sum(count_patches_train))
print('Number of images: ',len(count_patches_train))
print('Shape of all feature vectors combined: ',(feature_vectors_train.shape))

Number of feature vectors:  179674
Number of patches:  179674
Number of images:  150
Shape of all feature vectors combined:  (179674, 24)


In [7]:
kmeans = KMeans(n_clusters=32, random_state=0).fit(feature_vectors_train)

In [ ]:
# print(kmeans.labels_)
# print(kmeans.cluster_centers_)

In [51]:
# assigning each 24D feature vector of training set to a cluster
train_clustering = np.zeros((len(feature_vectors_train),1),dtype=int)
index = 0
for i in feature_vectors_train:
    distances =np.zeros((32,1))
    for j in range(32): # calculating distance of each 24D feature vector to every cluster
        distances[j] = distance(i, kmeans.cluster_centers_[j]) 
    cluster_id = np.argmin(distances,axis=0) # finding the cluster-id
    train_clustering[index] = cluster_id # storing the cluster-id for each 24D feature vector
    index +=1
    
# or
# train_clustering  = kmeans.predict(feature_vectors_train)

In [56]:
# Extracting Bag of Visual Words for images in training set
bovw_train =[]
index = 0
print(len(count_patches_train))
for i in range(len(count_patches_train)):
    bovw_img = [0]*32    
    clusters = train_clustering[index: index+count_patches_train[i]]
    index = index + count_patches_train[i]
    print("patches:",count_patches_train[i])
    print(len(clusters))
    unique, counts = np.unique(clusters, return_counts=True)
    dict_counts = dict(zip(unique, counts))
#     print(dict_counts)
    for j in range(32):
        bovw_img[j] = dict_counts.get(j,0)
#     print(bovw_img)
#     bovw_train.append(np.array(bovw_img))
    bovw_train.append(np.array(bovw_img)/count_patches_train[i])
    print(bovw_img)
    print(sum(bovw_img))
    print(" ")
bovw_train=np.array(bovw_train)
print(len(bovw_train))

150
patches: 425
425
[11, 69, 14, 68, 3, 7, 0, 47, 3, 0, 26, 0, 0, 0, 16, 0, 0, 25, 0, 0, 1, 14, 2, 0, 21, 9, 0, 76, 0, 12, 0, 1]
425
 
patches: 130
130
[1, 0, 8, 5, 2, 0, 7, 0, 0, 0, 7, 5, 5, 13, 0, 6, 0, 22, 0, 0, 0, 3, 19, 3, 1, 19, 0, 1, 0, 0, 0, 3]
130
 
patches: 672
672
[3, 5, 0, 4, 60, 11, 0, 69, 1, 0, 26, 66, 54, 11, 2, 3, 0, 101, 0, 11, 1, 142, 37, 5, 0, 1, 1, 31, 0, 27, 0, 0]
672
 
patches: 280
280
[20, 0, 35, 2, 13, 6, 6, 14, 0, 16, 0, 1, 2, 1, 18, 0, 14, 13, 2, 0, 1, 17, 16, 20, 4, 6, 10, 10, 0, 1, 24, 8]
280
 
patches: 1650
1650
[12, 4, 39, 16, 8, 938, 1, 9, 0, 38, 9, 3, 0, 0, 0, 18, 69, 41, 49, 0, 0, 57, 21, 7, 0, 0, 11, 0, 2, 225, 2, 71]
1650
 
patches: 80
80
[0, 1, 5, 0, 6, 16, 2, 0, 0, 0, 0, 6, 4, 0, 0, 0, 0, 4, 0, 0, 10, 13, 0, 0, 0, 1, 0, 0, 0, 10, 0, 2]
80
 
patches: 300
300
[6, 0, 7, 26, 1, 0, 1, 5, 6, 7, 5, 0, 31, 41, 5, 34, 5, 12, 1, 0, 0, 0, 16, 27, 27, 2, 2, 22, 0, 0, 0, 11]
300
 
patches: 1148
1148
[0, 28, 1, 0, 18, 9, 0, 0, 2, 0, 2, 451, 45, 0, 0, 0, 0, 67, 1

[2, 0, 37, 10, 27, 24, 2, 1, 0, 2, 2, 0, 2, 0, 2, 3, 4, 75, 2, 0, 1, 72, 5, 10, 1, 0, 0, 9, 0, 36, 0, 7]
336
 
patches: 2040
2040
[55, 50, 357, 43, 193, 123, 13, 24, 37, 27, 65, 17, 22, 36, 60, 15, 7, 134, 0, 41, 23, 317, 3, 35, 12, 39, 11, 67, 1, 197, 7, 9]
2040
 
patches: 96
96
[0, 3, 9, 0, 9, 0, 1, 0, 0, 0, 4, 2, 0, 1, 0, 0, 0, 19, 0, 1, 1, 37, 0, 2, 1, 0, 1, 0, 0, 5, 0, 0]
96
 
patches: 300
300
[0, 1, 35, 17, 21, 7, 2, 13, 4, 0, 8, 0, 1, 0, 1, 1, 0, 83, 0, 0, 0, 59, 1, 15, 3, 4, 0, 0, 0, 23, 0, 1]
300
 
patches: 280
280
[5, 0, 3, 4, 42, 1, 3, 4, 4, 9, 10, 6, 0, 0, 2, 0, 0, 63, 0, 0, 1, 39, 1, 52, 13, 0, 0, 2, 5, 8, 0, 3]
280
 
150


In [58]:
# Extracting 24-D feature vectors for testing data
feature_vectors_test = []
count_test = 0
count_patches_test = []
for img in test_images: # iterating over every image in testing set
    count_test += 1 
#     print(count_test)
    image = cv2.imread(img)  # reading the image
    shape = image.shape  # finding shape of the image
    size_1 = int(np.ceil(shape[0]/32)*32)  # finding the nearest integer to form 32x32 patches
    size_2 = int(np.ceil(shape[1]/32)*32)  # finding the nearest integer to form 32x32 patches 
    rf = [0,0]   # repeating_factor for padding image
    rf[0] = (size_1-shape[0])
    rf[1] = (size_2-shape[1])
    # padding the image by border pixel values on each side
    final_image = cv2.copyMakeBorder(image, rf[0]//2, rf[0]-rf[0]//2, rf[1]//2,rf[1]-rf[1]//2, cv2.BORDER_REPLICATE)
    patches = np.ndarray(shape=(size_1//32,size_2//32,32,32,3), dtype=np.uint8) # to store the patches
    # forming patches
    for i in range(size_1//32): # storing the patches
        for j in range(size_2//32):
            patches[i,j] = final_image[32*i:32*(i+1),32*j:32*(j+1)]
    # colour histogram for every patch
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            feature_vectors_test.append(colour_histogram(patches[i,j]))
    count_patches_test.append((size_1//32)*(size_2//32)) # storing the number of patches for each image
feature_vectors_test = np.array(feature_vectors_test)

In [59]:
print('Number of feature vectors: ',len(feature_vectors_test))
print('Number of patches: ',sum(count_patches_test))
print('Number of images: ',len(count_patches_test))
print('Shape of all feature vectors combined: ',(feature_vectors_test.shape))

Number of feature vectors:  159561
Number of patches:  159561
Number of images:  150
Shape of all feature vectors combined:  (159561, 24)


In [60]:
# assigning each 24D feature vector of test set to a cluster
test_clustering = np.zeros((len(feature_vectors_test),1),dtype=int)
index = 0
for i in feature_vectors_test:
    distances =np.zeros((32,1))
    for j in range(32): # calculating distance of each 24D feature vector to every cluster
        distances[j] = distance(i, kmeans.cluster_centers_[j]) 
    cluster_id = np.argmin(distances,axis=0) # finding the cluster-id
    test_clustering[index] = cluster_id # storing the cluster-id for each 24D feature vector
    index +=1
    
# or
# test_clustering  = kmeans.predict(feature_vectors_test)

In [67]:
# Extracting Bag of Visual Words for images in test set
bovw_test =[]
index = 0
print(len(count_patches_test))
for i in range(len(count_patches_test)):
    bovw_img = [0]*32    
    clusters = test_clustering[index: index+count_patches_test[i]]
    index = index + count_patches_test[i]
    print("patches:",count_patches_test[i])
    print(len(clusters))
    unique, counts = np.unique(clusters, return_counts=True)
    dict_counts = dict(zip(unique, counts))
#     print(dict_counts)
    for j in range(32):
        bovw_img[j] = dict_counts.get(j,0)
#     print(bovw_img)
#     bovw_test.append(np.array(bovw_img))
    bovw_test.append(np.array(bovw_img)/count_patches_test[i])
    print(bovw_img)
    print(sum(bovw_img))
    print(" ")
bovw_test=np.array(bovw_test)
print(len(bovw_test))

150
patches: 77
77
[0, 1, 1, 4, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 2, 9, 0, 0, 0, 16, 0, 0, 4, 0, 22]
77
 
patches: 117
117
[3, 0, 2, 4, 27, 0, 0, 7, 0, 2, 3, 0, 4, 0, 0, 1, 2, 20, 4, 1, 0, 6, 14, 2, 0, 0, 1, 9, 0, 1, 0, 4]
117
 
patches: 80
80
[1, 0, 0, 12, 0, 0, 0, 19, 0, 0, 0, 0, 3, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 35, 0, 2, 0, 0, 0, 0]
80
 
patches: 70
70
[0, 0, 15, 0, 10, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 16, 0, 0, 5, 3, 6, 6, 0, 0, 2, 1, 0, 0, 1, 0]
70
 
patches: 70
70
[0, 0, 1, 0, 1, 17, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 2, 10, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0]
70
 
patches: 374
374
[19, 0, 34, 22, 33, 0, 10, 19, 10, 3, 8, 0, 3, 0, 1, 2, 3, 68, 6, 0, 0, 17, 6, 21, 16, 15, 5, 21, 0, 10, 3, 19]
374
 
patches: 1900
1900
[772, 7, 11, 9, 32, 0, 10, 92, 19, 24, 21, 66, 16, 1, 48, 23, 177, 48, 1, 257, 0, 4, 55, 1, 19, 61, 13, 99, 0, 2, 9, 3]
1900
 
patches: 285
285
[5, 0, 29, 11, 30, 0, 8, 25, 2, 0, 10, 0, 14, 0, 6, 3, 19, 26, 0, 5, 0, 0, 21, 0, 5, 0, 13, 5

In [72]:
path = 'C:/Users/Tirthashree/Documents/MTech/Notes/DLA/Assignments/Group20/Classification/Image_Group20/'
os.chdir(path)

# df = pd.DataFrame(feature_vectors_train) 
# df.to_csv("train24D.csv",header=False,index=False)

# df = pd.DataFrame(feature_vectors_test) 
# df.to_csv("test24D.csv",header=False,index=False)

df = pd.DataFrame(bovw_train) 
df.to_csv("trainBoVW.csv",header=False,index=False)

df = pd.DataFrame(bovw_test) 
df.to_csv("testBoVW.csv",header=False,index=False)